In [ ]:
import nltk
import re
import os
import pandas as pd
import numpy as np
import scipy.sparse as sp
import lightgbm as lgb
import scipy.sparse as sp

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, classification_report
from urllib.parse import unquote, urlparse, quote
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 数据读取与拼接
train = pd.read_csv('./data/dataTrain.csv')
test = pd.read_csv('./data/dataA.csv')
train['f3'] = train['f3'].map({'low': 0, 'mid': 1, 'high': 2})
test['f3'] = test['f3'].map({'low': 0, 'mid': 1, 'high': 2})

# 剔除最后1w
train = train[:50000]
sum(train['label'] == 1)

12470

In [ ]:
allf = [i for i in train.columns if i not in ['id', 'label']]
for f in allf:
  nzeros = train[f] != 0
  zeros = train[f] == 0
  label_zeros = train['label'] == 0
  label_nzeros = train['label'] != 0
  print(f)
  print(f'!0: {sum(nzeros)} ; =0:{sum(zeros)}')
  print(f'0&0:{sum(zeros&label_zeros)} 0&1:{sum(zeros&label_nzeros)} 1&0:{sum(nzeros&label_zeros)} 1&1:{sum(nzeros&label_nzeros)}')

f1
!0: 15314 ; =0:34686
0&0:22791 0&1:11895 1&0:14739 1&1:575
f2
!0: 34359 ; =0:15641
0&0:8734 0&1:6907 1&0:28796 1&1:5563
f3
!0: 33618 ; =0:16382
0&0:13370 0&1:3012 1&0:24160 1&1:9458
f4
!0: 10765 ; =0:39235
0&0:34579 0&1:4656 1&0:2951 1&1:7814
f5
!0: 19658 ; =0:30342
0&0:27594 0&1:2748 1&0:9936 1&1:9722
f6
!0: 29068 ; =0:20932
0&0:19718 0&1:1214 1&0:17812 1&1:11256
f7
!0: 9755 ; =0:40245
0&0:30956 0&1:9289 1&0:6574 1&1:3181
f8
!0: 3074 ; =0:46926
0&0:35805 0&1:11121 1&0:1725 1&1:1349
f9
!0: 581 ; =0:49419
0&0:37010 0&1:12409 1&0:520 1&1:61
f10
!0: 857 ; =0:49143
0&0:36799 0&1:12344 1&0:731 1&1:126
f11
!0: 13 ; =0:49987
0&0:37520 0&1:12467 1&0:10 1&1:3
f12
!0: 109 ; =0:49891
0&0:37452 0&1:12439 1&0:78 1&1:31
f13
!0: 351 ; =0:49649
0&0:37266 0&1:12383 1&0:264 1&1:87
f14
!0: 17 ; =0:49983
0&0:37514 0&1:12469 1&0:16 1&1:1
f15
!0: 121 ; =0:49879
0&0:37433 0&1:12446 1&0:97 1&1:24
f16
!0: 2 ; =0:49998
0&0:37528 0&1:12470 1&0:2 1&1:0
f17
!0: 325 ; =0:49675
0&0:37291 0&1:12384 1&0:239 1&1:86


In [ ]:
idx = (train['f40'] != 0) #& (train['label'] == 1)
print(sum(train['f40'] != 0))
train[idx][['f40','label']]

5


,f40,label
4536,42,0
31350,177,0
38234,216,0
40788,189,0
49947,66,0


In [ ]:
idx = (train['f41'] != 0) #& (train['label'] == 1)
print(sum(train['f41'] != 0))
train[idx][['f41','label']]

1


,f41,label
14377,126,0


In [ ]:
idx = (train['f14'] != 0) #& (train['label'] == 1)
print(sum(train['f14'] != 0))
train[idx][['f14','label']]

17


,f14,label
1182,258,0
5178,39,1
5238,66,0
7386,126,0
10927,12,0
19478,57,0
20637,132,0
20853,15,0
25491,138,0
28693,162,0


In [ ]:
idx = (train['f16'] != 0) #& (train['label'] == 1)
print(sum(train['f16'] != 0))
train[idx][['f16','label']]

2


,f16,label
1367,36,0
7542,12,0


In [ ]:
idx = (train['f13'] != 0) #& (train['label'] == 1)
print(sum(train['f13'] != 0))
train[idx][['f13','label']]

351


,f13,label
50,93,0
137,24,0
166,30,0
335,18,0
414,9,1
...,...,...
49652,24,0
49665,81,0
49794,45,0
49897,24,0


In [ ]:
idx = (train['f27'] != 0) #& (train['label'] == 1)
print(sum(train['f27'] != 0))
train[idx][['f27','label']]

5


,f27,label
33513,21,0
34647,15,1
44654,18,0
46168,36,1
47848,21,1
